In [293]:
#import torch
import pandas as pd
import shapely
import numpy as np
import sys
#import torch
import argparse
import random
import math
import os
import warnings
from gensim.models import word2vec
from shapely.geometry import LineString, Polygon
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler # 好处在于可以保存训练集中的参数（均值、方差）
from sklearn.cluster import MeanShift
from scipy.stats import stats
#from torch.utils.data import DataLoader
#from torch.optim import lr_scheduler
from tqdm import tqdm
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 999
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

### 找Teile间的相似性的方法可以使用association rule吗，如果可以，应该如何从association rule生成对应的sentence。感觉这会是一个不错的题目 ------------- 即商业产品的word embedding

# Parts embedding
Before doing the embedding, we should be clear, what kind of features do we need for the emebdding voctor.<br>
like the word embedding, we train them with sentence, we need to create some 'sentence'. and the parts in one 
sentence should have some kinds of relationship.<br>
另外还应该考虑的是：用什么embedding的方法， 因为不同embedding的方法，得到的结果是不同的。

## Create sentence Corpus

In [2]:
class Create_sentence:
    def __init__(self, dict_parts, n = 10000):
        """
        dict_parts, type of dictionary
            store the information of each parts. similar parts are save under one key
        n, type of int
            set the total number of sentences that you want to generate
        """
        self.dict_parts = dict_parts
        self.n = n
    def __iter__(self):
        for i in range(n):
            # choose a key randomly
            key = random.choice(dict_parts.keys())
            value = random[key]
            yield self._create_sentence_form_words(value)
    def _create_sentence_from_words(value):
        #TODO
        
        pass

## Read data

In [348]:
data = pd.read_pickle('../results/all_norm_feature.pkl')

In [323]:
data.head()

,Jobid,Rot,Left down,right down,score,bounding box,Shapely Polygon,Area,length,area quote,convex area quote,convex area,area verhaltnis,centroid x/width,centroid y/height,width/height,num_of_corr,r1,r2,A/L,Left_down_x,Left_down_y,Right_down_y,norm_area,norm_left_down_x,norm_left_down_y,norm_right_down_y
0,792-0_10_0,90.000,"(93.0, 1.0)","(93.0, 377.0)",0.000,"(0.0, 0.0)","POLYGON ((159 49, 167 44, 168 44, 308 44, 308 ...",20525.000,0.847,-0.514,0.512,1.266,-0.965,0.733,2.238,0.560,0.910,0.000,1.000,0.315,93.000,1.000,377.000,0.916,-1.286,-1.994,-1.400
1,792-0_10_1,0.000,"(213.0, 96.0)","(213.0, 254.0)",4.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,1.000,0.000,0.239,213.000,96.000,254.000,0.608,-0.992,-1.772,-1.671
2,792-0_10_2,90.000,"(139.0, 311.0)","(139.0, 494.0)",4.000,"(0.0, 0.0)","POLYGON ((20 138, 19 138, 17 138, 16 138, 0 13...",12481.500,0.339,-1.070,-1.456,-0.100,-0.364,-1.348,-0.681,-0.503,0.008,0.000,1.000,-0.461,139.000,311.000,494.000,-0.163,-1.173,-1.271,-1.142
3,792-0_10_3,0.000,"(234.0, 453.0)","(234.0, 581.0)",4.000,"(0.0, 0.0)","POLYGON ((150 128, 149 128, 149 92, 149 91, 12...",14630.000,-0.032,-1.078,-1.227,0.252,-0.577,0.396,-1.971,-0.316,0.409,1.000,0.000,0.322,234.000,453.000,581.000,0.125,-0.940,-0.939,-0.950
4,792-0_10_4,90.000,"(164.0, 678.0)","(164.0, 890.0)",0.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,0.000,1.000,0.239,164.000,678.000,890.000,0.608,-1.112,-0.414,-0.269


## PE for features based structure
- create cluster based on different features and create sentences based on cluster.
    - which is more important, the cluster with one features or the cluster with more features.

In [148]:
# define method to get the n combination.
def get_combinations(nc, seq, si = 0, com = []):
    """
    input:
    nc, type of int
        set the number of items in one combination
    seq, type of list
        from whom we get the items
    si, type of int
        only consider the items which index larger than si
    com, type of list
        used for recursion, should be set to [] when calling the function
    """
    out = []
    for i in range(si, len(seq)):
        if seq[i] not in com:
            com.append(seq[i])
        else:
            continue
        if len(com) < nc:
            com, tmp_out = get_combinations(nc, seq, i, com)
            out.extend(tmp_out)
        elif len(com) == nc:
            #print(com)
            out.append(com)
            com = com[:-1]
        else:
            print('ERROR!!!')
            #print(com)
    return com[:-1], out

In [324]:
# feature selection
df = data[['norm_area', 'length', 'area quote', 'convex area quote', 'convex area', 'area verhaltnis', 'centroid x/width',
        'centroid y/height', 'num_of_corr', 'A/L']]
df = df.drop_duplicates()
df['type'] = df.index

In [152]:
# set container and meanshift cluster model
from sklearn.cluster import MeanShift
df_cluster = pd.DataFrame(df['type'], index=df.index)
ms = MeanShift()

In [155]:
# level one
_, items = get_combinations(1, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 11/11 [00:01<00:00,  9.24it/s]


In [156]:
#level two
_, items = get_combinations(2, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 55/55 [00:07<00:00,  8.26it/s]


In [157]:
#level three
_, items = get_combinations(3, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 165/165 [00:21<00:00,  6.55it/s]


In [158]:
#level four
_, items = get_combinations(4, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 330/330 [00:41<00:00,  8.65it/s]


In [159]:
#level five
_, items = get_combinations(5, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 462/462 [01:00<00:00,  8.65it/s]


In [160]:
#level six
_, items = get_combinations(6, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 462/462 [00:54<00:00,  8.67it/s]


In [219]:
#level seven
_, items = get_combinations(7, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 330/330 [00:38<00:00,  8.73it/s]


In [220]:
#level eifht
_, items = get_combinations(8, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 165/165 [00:20<00:00,  7.80it/s]


In [221]:
#level night
_, items = get_combinations(9, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 55/55 [00:07<00:00,  8.36it/s]


In [222]:
#level ten
_, items = get_combinations(10, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 11/11 [00:01<00:00,  8.16it/s]


In [223]:
#level eleven
_, items = get_combinations(11, df.columns, si = 0, com = [])
for col in tqdm(items):
    if col == 'type':
        continue
    clustering = ms.fit(df[col])
    df_cluster.loc[:, 'clustering after ' + str(col)] = clustering.labels_

100%|██████████| 1/1 [00:00<00:00,  6.19it/s]


In [224]:
# only take the cluster which menge between 10 and 30
out = []
df_cluster.apply(lambda x: [out.append(df['type'][x == i]) for i, j in zip(x.value_counts().index, x.value_counts().tolist()) if j < 30 and j > 10])

type                                                                                                                                                                                                    []
clustering after ['norm_area']                                                                                                                                                          [None, None, None]
clustering after ['length']                                                                                                                                                                             []
clustering after ['area quote']                                                                                                                                                                     [None]
clustering after ['convex area quote']                                                                                                                                                      

In [272]:
# change data type
out = [i.tolist() for i in out]

In [296]:
# create sentences from the cluster
sentences = []
for i in tqdm(out):
    num_sents = random.randint(5, 20)
    len_sents = random.randint(min(len(i), 5), min(len(i), 20))
    counter = 0
    for j in range(num_sents):
        tmp = []
        counter = 0
        while counter < len_sents:
            chosen = random.choice(i)
            if str(chosen) in tmp:
                continue
            tmp.append(str(chosen))
            counter = counter + 1
        sentences.append(tmp)


100%|██████████| 1112/1112 [00:00<00:00, 2597.44it/s]

[[7,
  11,
  12,
  14,
  15,
  16,
  17,
  20,
  24,
  26,
  27,
  29,
  33,
  39,
  42,
  44,
  45,
  46,
  51,
  97,
  112,
  137,
  216],
 [0, 1, 5, 6, 9, 28, 32, 36, 43, 49, 53, 57, 76, 77, 81, 114, 171, 243],
 [2, 3, 10, 22, 23, 25, 31, 37, 38, 59, 67, 73, 103],
 [0, 1, 2, 3, 6, 9, 10, 12, 14, 15, 17, 24, 33, 44, 57, 81, 112, 114, 243],
 [0,
  1,
  5,
  11,
  14,
  16,
  17,
  22,
  28,
  29,
  33,
  37,
  42,
  45,
  46,
  49,
  59,
  67,
  73,
  76,
  81,
  97,
  103,
  243],
 [9, 12, 15, 20, 24, 25, 27, 31, 32, 36, 38, 43, 57, 77, 112, 216],
 [7,
  11,
  12,
  14,
  15,
  16,
  17,
  20,
  23,
  24,
  26,
  27,
  29,
  33,
  38,
  42,
  44,
  45,
  46,
  51,
  59,
  73,
  97,
  103,
  112,
  137,
  216],
 [0,
  1,
  2,
  3,
  5,
  6,
  9,
  10,
  22,
  25,
  28,
  31,
  32,
  36,
  37,
  39,
  43,
  49,
  57,
  67,
  76,
  77,
  114],
 [0,
  2,
  3,
  9,
  10,
  12,
  14,
  15,
  16,
  17,
  20,
  24,
  31,
  32,
  43,
  44,
  45,
  49,
  51,
  57,
  73,
  76,
  81,
  97,
  112

## Train word2vec model

In [306]:
# train and test the word Embedding
# sentences = word2vec.Text8Corpus('sss.txt')
model = word2vec.Word2Vec(sentences, min_count = 5, size = 8)

## Test the model with similarity

In [327]:
df['wv'] = df['type'].map(lambda x: model.wv[str(x)])

In [328]:
model.wv['2']

array([-0.4064451 , -0.14149065,  1.0258222 ,  1.0350567 ,  0.6432764 ,
        2.049479  , -0.96673936,  0.31581807], dtype=float32)

In [351]:
# merge back to dataframe on length
a = data.merge(df[['length', 'wv']], how = 'left', on = 'length')

In [345]:
for i in model.most_similar('3', topn = 50):
    print(i)

('10', 0.9938647150993347)
('243', 0.9927123188972473)
('53', 0.9915392994880676)
('39', 0.9899251461029053)
('2', 0.9895091652870178)
('171', 0.9883044958114624)
('23', 0.9879540801048279)
('33', 0.9847519993782043)
('6', 0.9842872619628906)
('7', 0.9832440614700317)
('216', 0.9830256700515747)
('137', 0.981259822845459)
('51', 0.9788312315940857)
('17', 0.9782271385192871)
('15', 0.977448582649231)
('20', 0.9770758152008057)
('12', 0.9758028388023376)
('26', 0.9745304584503174)
('9', 0.974463939666748)
('45', 0.9737380743026733)
('24', 0.9734922051429749)
('32', 0.9712787866592407)
('44', 0.970125675201416)
('0', 0.96909099817276)
('16', 0.9671755433082581)
('11', 0.9618031978607178)
('27', 0.9614555835723877)
('14', 0.9602183699607849)
('29', 0.9502819180488586)
('42', 0.948022723197937)
('36', 0.9412325620651245)
('1', 0.9391903877258301)
('49', 0.9379599094390869)
('43', 0.9373362064361572)
('46', 0.9362534284591675)
('31', 0.9187831878662109)
('38', 0.9141314625740051)
('25', 0.9

In [349]:
data.head()

,Jobid,Rot,Left down,right down,score,bounding box,Shapely Polygon,Area,length,area quote,convex area quote,convex area,area verhaltnis,centroid x/width,centroid y/height,width/height,num_of_corr,r1,r2,A/L,Left_down_x,Left_down_y,Right_down_y,norm_area,norm_left_down_x,norm_left_down_y,norm_right_down_y
0,792-0_10_0,90.000,"(93.0, 1.0)","(93.0, 377.0)",0.000,"(0.0, 0.0)","POLYGON ((159 49, 167 44, 168 44, 308 44, 308 ...",20525.000,0.847,-0.514,0.512,1.266,-0.965,0.733,2.238,0.560,0.910,0.000,1.000,0.315,93.000,1.000,377.000,0.916,-1.286,-1.994,-1.400
1,792-0_10_1,0.000,"(213.0, 96.0)","(213.0, 254.0)",4.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,1.000,0.000,0.239,213.000,96.000,254.000,0.608,-0.992,-1.772,-1.671
2,792-0_10_2,90.000,"(139.0, 311.0)","(139.0, 494.0)",4.000,"(0.0, 0.0)","POLYGON ((20 138, 19 138, 17 138, 16 138, 0 13...",12481.500,0.339,-1.070,-1.456,-0.100,-0.364,-1.348,-0.681,-0.503,0.008,0.000,1.000,-0.461,139.000,311.000,494.000,-0.163,-1.173,-1.271,-1.142
3,792-0_10_3,0.000,"(234.0, 453.0)","(234.0, 581.0)",4.000,"(0.0, 0.0)","POLYGON ((150 128, 149 128, 149 92, 149 91, 12...",14630.000,-0.032,-1.078,-1.227,0.252,-0.577,0.396,-1.971,-0.316,0.409,1.000,0.000,0.322,234.000,453.000,581.000,0.125,-0.940,-0.939,-0.950
4,792-0_10_4,90.000,"(164.0, 678.0)","(164.0, 890.0)",0.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,0.000,1.000,0.239,164.000,678.000,890.000,0.608,-1.112,-0.414,-0.269


In [352]:
a.head()

,Jobid,Rot,Left down,right down,score,bounding box,Shapely Polygon,Area,length,area quote,convex area quote,convex area,area verhaltnis,centroid x/width,centroid y/height,width/height,num_of_corr,r1,r2,A/L,Left_down_x,Left_down_y,Right_down_y,norm_area,norm_left_down_x,norm_left_down_y,norm_right_down_y,wv
0,792-0_10_0,90.000,"(93.0, 1.0)","(93.0, 377.0)",0.000,"(0.0, 0.0)","POLYGON ((159 49, 167 44, 168 44, 308 44, 308 ...",20525.000,0.847,-0.514,0.512,1.266,-0.965,0.733,2.238,0.560,0.910,0.000,1.000,0.315,93.000,1.000,377.000,0.916,-1.286,-1.994,-1.400,"[-0.40866196, 0.133643, 1.0462157, 0.6165552, ..."
1,792-0_10_1,0.000,"(213.0, 96.0)","(213.0, 254.0)",4.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,1.000,0.000,0.239,213.000,96.000,254.000,0.608,-0.992,-1.772,-1.671,"[-0.52308816, 0.15136395, 1.2147502, 0.7406462..."
2,792-0_10_2,90.000,"(139.0, 311.0)","(139.0, 494.0)",4.000,"(0.0, 0.0)","POLYGON ((20 138, 19 138, 17 138, 16 138, 0 13...",12481.500,0.339,-1.070,-1.456,-0.100,-0.364,-1.348,-0.681,-0.503,0.008,0.000,1.000,-0.461,139.000,311.000,494.000,-0.163,-1.173,-1.271,-1.142,"[-0.4064451, -0.14149065, 1.0258222, 1.0350567..."
3,792-0_10_3,0.000,"(234.0, 453.0)","(234.0, 581.0)",4.000,"(0.0, 0.0)","POLYGON ((150 128, 149 128, 149 92, 149 91, 12...",14630.000,-0.032,-1.078,-1.227,0.252,-0.577,0.396,-1.971,-0.316,0.409,1.000,0.000,0.322,234.000,453.000,581.000,0.125,-0.940,-0.939,-0.950,"[-0.38796213, 0.13429478, 0.9971617, 0.8649837..."
4,792-0_10_4,90.000,"(164.0, 678.0)","(164.0, 890.0)",0.000,"(0.0, 0.0)","POLYGON ((199 158, 93 158, 66 132, 0 132, 0 13...",18230.000,0.580,-0.789,0.560,1.192,-1.334,-1.092,0.347,-0.497,-0.493,0.000,1.000,0.239,164.000,678.000,890.000,0.608,-1.112,-0.414,-0.269,"[-0.52308816, 0.15136395, 1.2147502, 0.7406462..."


## PE for hierarchies structure

## PE for association based structure

## PE ohne structure